In [ ]:
!pip install openimages
!pip install --upgrade pip
!pip install torch
!pip install torchvision
!pip install glob
!pip install PIL
!pip install numpy
!pip install itertools

In [74]:
import torch
import glob
import numpy as np
import itertools 
import os

from torchvision import datasets
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from skimage.transform import resize
from openimages.download import download_dataset
from PIL import Image
from torchvision.models import resnet152, ResNet152_Weights
from openimages.download import download_dataset

In [ ]:
if not os.path.exists('data'):
    os.makedirs('data')

#  Atsiunciame paveikslelius
classes = ["Cat", "Dog", "Lamp"]
data_directory = "data"
classes_file_path = "data/classes.txt"
number_for_samples = 300
download_dataset(data_directory, classes, limit=number_for_samples)

In [8]:
#Su class file 
with open(classes_file_path) as cf:
  class_list = [line.split('\n')[0].lower().split(" ") for line in cf.readlines()] #pasplitinam tas klases, nes kaikurios yra iš dviejų žodžių, kad galimą būtų sudėti į array patogiau

class_indexes = { c: [idx for idx, s in enumerate(class_list) if c.lower() in s][0] for c in classes } #Paprašome, kad surastų kur prasideda kiekvienos atskiros klasės indeksas
print(class_indexes)

{'Cat': 282, 'Dog': 151, 'Lamp': 846}


In [9]:
weights = ResNet152_Weights.DEFAULT
model = resnet152(weights=weights)
model.eval()

preprocess = weights.transforms()

In [42]:
class CustomLab1Dataset(Dataset):
    def __init__(self, images_dir, preprocess):
        self.images_dir = images_dir
        self.preprocess = preprocess

        self.class1_files = glob.glob(self.images_dir + "/{}/images/*.jpg".format(classes[0].lower()))
        self.class2_files = glob.glob(self.images_dir + "/{}/images/*.jpg".format(classes[1].lower()))
        self.class3_files = glob.glob(self.images_dir + "/{}/images/*.jpg".format(classes[2].lower()))

        self.class1 = len(self.class1_files)
        self.class2 = len(self.class2_files)
        self.class3 = len(self.class3_files)

        self.files = self.class1_files + self.class2_files + self.class3_files

        self.labels = np.zeros(len(self.files))
        self.labels[self.class1:self.class1+self.class2] = 1
        self.labels[self.class1+self.class2:] = 2

        self.order =  [x for x in np.random.permutation(len(self.labels))]
        self.files = [self.files[x] for x in self.order]
        self.labels = [self.labels[x] for x in self.order]


    def __len__(self):
        return (len(self.files))

    def __getitem__(self, index):

        file = self.files[index]

        image = Image.open(file).convert('RGB')
        process_image = self.preprocess(image)

        label = self.labels[index]
        return (process_image, label)

In [71]:
dataset = CustomLab1Dataset("./data", preprocess)
dataLoader = torch.utils.data.DataLoader(dataset, batch_size=32, num_workers=0, shuffle=True)

In [68]:
predictions, actual_values = {c: [] for c in classes}, {c: [] for c in classes}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #pas mane su  CPU, bet per google colab veiktų

with torch.no_grad():
    for images, labels in dataLoader:
        images = images.to(device)
        outputs = model(images).softmax(dim=1)

        
        for idx, c in enumerate(classes):
            class_probabilities = outputs[:, class_indexes[c]].detach().cpu().numpy()
            predictions[c].extend(class_probabilities)
            actual_values[c].extend(labels.cpu().numpy() == idx)

In [69]:
from sklearn.metrics import confusion_matrix, classification_report

def evaluate_model_performance(predicted_scores, true_labels, threshold=0.5):

    binary_predictions = (predicted_scores >= threshold).astype(int)

    confusion_mtx = confusion_matrix(true_labels, binary_predictions)
    
    tp = confusion_mtx[1, 1]
    tn = confusion_mtx[0, 0]
    fp = confusion_mtx[0, 1]
    fn = confusion_mtx[1, 0]

    print(f"True Positives (TP): {tp}, True Negatives (TN): {tn}, False Positives (FP): {fp}, False Negatives (FN): {fn}")

    # Dedu į biblioteką, kaip jūs rekomendavote
    performance_metrics = {}
    performance_metrics["accuracy"] = (tp + tn) / (tp + tn + fp + fn)
    performance_metrics["sensitivity"] = tp / (tp + fn) # Recall
    performance_metrics["precision"] = tp / (tp + fp)
    performance_metrics["F1"] = 2 * (performance_metrics["precision"] * performance_metrics["sensitivity"]) / (performance_metrics["precision"] + performance_metrics["sensitivity"])

    # Display classification report for further insights
    print("\nResults:")
    print(classification_report(true_labels, binary_predictions))

    return performance_metrics


In [73]:
performance_summary = {}
performance_summary['average'] = {}
for cl in classes:
    print(f"\nEvaluating performance for '{cl}' class:")
    performance_summary[cl] = evaluate_model_performance(np.array(predictions[cl]), np.array(actual_values[cl]))

    for metric, value in performance_summary[cl].items():
        print(f"{metric.capitalize()}: {value}")
        
        if metric in performance_summary['average']:
            performance_summary['average'][metric] += value
        else:
            performance_summary['average'][metric] = value


for metric in performance_summary['average']:
    performance_summary['average'][metric] /= len(classes)


Evaluating performance for class 'Cat':
True Positives: 8, True Negatives: 600, False Positives: 0, False Negatives: 292

Classification Report:
              precision    recall  f1-score   support

       False       0.67      1.00      0.80       600
        True       1.00      0.03      0.05       300

    accuracy                           0.68       900
   macro avg       0.84      0.51      0.43       900
weighted avg       0.78      0.68      0.55       900

Accuracy: 0.6755555555555556
Sensitivity: 0.02666666666666667
Precision: 1.0
F1: 0.05194805194805195

Evaluating performance for class 'Dog':
True Positives: 3, True Negatives: 600, False Positives: 0, False Negatives: 297

Classification Report:
              precision    recall  f1-score   support

       False       0.67      1.00      0.80       600
        True       1.00      0.01      0.02       300

    accuracy                           0.67       900
   macro avg       0.83      0.51      0.41       900
weighted

In [26]:
import requests
from PIL import Image
from io import BytesIO

def classify_image_from_url(image_url, model, preprocess):
    
    response = requests.get(image_url)
    image = Image.open(BytesIO(response.content)).convert('RGB')

    
    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0)  # mini bečas

    # pas mane defoltiniu pajungia ne su kuda o su mano CPU ir man neveikia modeliai su AMD plokšte, bet jei per google colab leisčiau - veiktų
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)
    
    probabilities = torch.nn.functional.softmax(output[0], dim=0)

    _, top_catid = torch.topk(probabilities, 1)

    classifier_number = top_catid.item() #Paprašau rezultato indekso classes.txt faile

    print(classifier_number)

    if classifier_number >= class_indexes['Dog'] and classifier_number < class_indexes['Cat']:
        return "Dog"
    elif classifier_number >= class_indexes['Cat'] and classifier_number < 294: 
        return "Cat"
    elif classifier_number == class_indexes['Lamp']:
        return "Lamp"
    else:
        return "Unknown"

In [27]:
image_url = "https://www.theenglishhome.co.uk/wp-content/uploads/2023/10/Cathy-Nordstrom-x-Salig-Studio-Living-room-Florence-in-Tobacco-base-in-Mist-Mikael-Lundblad-Portrait.jpg"
classification_result = classify_image_from_url(image_url, model, preprocess)
print(f"The image is a {classification_result}.")

846
The image is a Lamp.
